In [186]:
import glob
import itertools
import os
import os.path
import random
import binascii

In [143]:
corpus_folder = "./data/test"
test_file_path = "./data/test/1.txt"

In [155]:
def file_to_shingles(file_path, shingle_size=2, do_hash=True):
    with open(file_path, 'r') as infile:
        content = infile.read()
        shingles = set()
        for i in range(len(content) - shingle_size):
            shingle = content[i:i+shingle_size]
            shingles.add(
                hash_shingle(shingle) if do_hash else shingle
            )
        return sorted(list(shingles))

def hash_shingle(shingle):
    """Hashes a shingle to a 32bit integer.
    
    """
    return binascii.crc32(shingle.encode("utf-8")) & 0xffffffff

for shingle in file_to_shingles(test_file_path):
    print(shingle)

if len(file_to_shingles(test_file_path)) != len(file_to_shingles(test_file_path, do_hash=False)):
    print("Hey, we have a problem")

15075007
105998545
156340709
238835196
312708591
491853531
809500611
996668263
1648209365
2428467314
2563917151
2726894320
2873054939
2891092674
3478361055
3523499602
3761386704
3881708472


In [156]:
def jacard_similarity(set_1, set_2):
    return len(set_1.intersection(set_2)) / len(set_1.union(set_2))

jacard_similarity(set(["a","b","c"]), set(["b","c","d"]))

0.5

In [182]:
# next prime of 2^32 - 1 is 4294967311
# http://compoasso.free.fr/primelistweb/page/prime/liste_online_en.php
        
class HashStore:
    max_shingle_hash = 2**32 - 1
    next_prime = 4294967311
    
    def __init__(self, number_of_hash_functions=100, seed=None):
        if seed is not None:
            random.seed(seed)
        self.n_hash_functions = number_of_hash_functions  # the length of the signature
        self.a_coefficients = self.generate_coefficient_list(number_of_hash_functions)
        self.b_coefficients = self.generate_coefficient_list(number_of_hash_functions)
    
    def generate_coefficient_list(self, number_of_coefficients):
        # integers in python have arbitrary precision (not applicable to numpy)
        # https://mortada.net/can-integer-operations-overflow-in-python.html
        coefficients = []
        
        def random_int():
            return random.randint(1, self.max_shingle_hash)
        
        for _ in range(number_of_coefficients):
            candidate = random_int()
            while candidate in coefficients:
                candidate = random_int()
            coefficients.append(candidate)
        
        return coefficients
    
    def hash_function(self, function_index, value):
        assert function_index < self.n_hash_functions, "Function index out of range"
        assert type(value) == int, "Cannot hash strings, yet..."
        a = self.a_coefficients[function_index]
        b = self.b_coefficients[function_index]
        return (a * value + b) % self.next_prime & 0xffffffff  # not really needed it
    
    def get_signature(self, set_of_shingles):
        signature = []  # needs to keep the order of insertion, the set will not grant it
        for i in range(self.n_hash_functions):
            signature.append(
                min(map(lambda x: self.hash_function(i,x) ,set_of_shingles))
            )
        return signature
    
            
    
shingles_set = file_to_shingles(test_file_path)

hash_store = HashStore(3, seed=42)
hash_store.hash_function(2,4)
hash_store.get_signature(shingles_set)


[74223370, 45489653, 215062606]

In [229]:
hash_store = HashStore(4, seed=42)
file_list = glob.glob(os.path.join(corpus_folder,"*"))
shingle_sets = list(map(file_to_shingles, file_list))
signature_matrix = list(map(hash_store.get_signature , shingle_sets))
signature_matrix  # aka list of lists

[[120392129, 281924773, 48793314, 113009555],
 [816028170, 63038758, 121942940, 227166117],
 [247472466, 63038758, 36547400, 54618441]]

In [210]:
asd = ["a","b","c","d"]
list(itertools.combinations(asd,2))

[('a', 'b'), ('a', 'c'), ('a', 'd'), ('b', 'c'), ('b', 'd'), ('c', 'd')]

In [212]:
for set1, set2 in itertools.combinations(shingle_sets,2):
#     print(set1,set2)
    print(jacard_similarity(set(set1),set(set2)))  

0.11428571428571428
0.11428571428571428
0.6363636363636364


In [221]:
sign1, sign2 = signature_matrix[0], signature_matrix[2]

def compare_signatures(sign1, sign2):
    assert len(sign1) == len(sign2)
    return sum([x == y for x,y in zip(sign1, sign2)]) / len(sign1)

compare_signatures(sign1,sign2)

0.2

In [228]:
for sign1, sign2 in itertools.combinations(signature_matrix,2):
#     print(set1,set2)
    print(compare_signatures(sign1,sign2))  

0.0
0.0
0.25
